# Lab Exercise: SQL Analysis with Polars

In this lab, you'll practice SQL queries using Polars' built-in SQL functionality. Complete each exercise by writing the appropriate SQL query.

In [ ]:
# Setup - Run this cell first
import polars as pl

# Load data
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv', null_values='NA')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv', null_values='NA')
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv', null_values='NA')
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv', null_values='NA')
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv', null_values='NA', ignore_errors=True)

flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))

# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


## Exercise 1: Basic Queries

### 1.1 Find all unique carriers in the airlines table

In [ ]:
result = ctx.execute("""
SELECT DISTINCT carrier
FROM airlines
""")

print(result)


shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


### 1.2 Find the top 10 destinations by number of flights

In [ ]:
result = ctx.execute("""
SELECT dest, COUNT(*) AS num_flights
FROM flights
GROUP BY dest
ORDER BY num_flights DESC
LIMIT 10
""")

print(result)

shape: (10, 2)
┌──────┬─────────────┐
│ dest ┆ num_flights │
│ ---  ┆ ---         │
│ str  ┆ u32         │
╞══════╪═════════════╡
│ ORD  ┆ 17283       │
│ ATL  ┆ 17215       │
│ LAX  ┆ 16174       │
│ BOS  ┆ 15508       │
│ MCO  ┆ 14082       │
│ CLT  ┆ 14064       │
│ SFO  ┆ 13331       │
│ FLL  ┆ 12055       │
│ MIA  ┆ 11728       │
│ DCA  ┆ 9705        │
└──────┴─────────────┘


### 1.3 Find all flights that departed more than 2 hours late (120 minutes)

In [ ]:
result = ctx.execute("""
SELECT *
FROM flights
WHERE dep_delay > 120
""")

print(result)

shape: (9_723, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01 23:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 957      ┆ … ┆ 200      ┆ 7    ┆ 33     ┆ 2013-01-01 12:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1114     ┆ … ┆ 1416     ┆ 9    ┆ 0      ┆ 2013-01-01 14:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1540     ┆ … ┆ 1598     ┆ 13   ┆ 38     ┆ 2013-01-01 18:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1815     ┆ … ┆ 1134     ┆ 13   ┆ 25     ┆ 2013-01-01 18:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …     

## Exercise 2: Aggregation

### 2.1 Calculate the average departure delay for each origin airport

In [ ]:
result = ctx.execute("""
SELECT origin, AVG(dep_delay) AS avg_dep_delay
FROM flights
GROUP BY origin
""")

print(result)


shape: (3, 2)
┌────────┬───────────────┐
│ origin ┆ avg_dep_delay │
│ ---    ┆ ---           │
│ str    ┆ f64           │
╞════════╪═══════════════╡
│ EWR    ┆ 15.107954     │
│ JFK    ┆ 12.112159     │
│ LGA    ┆ 10.346876     │
└────────┴───────────────┘


### 2.2 Find the busiest month of the year

Count the number of flights per month and find which month has the most flights.

In [ ]:
result = ctx.execute("""
SELECT month, COUNT(*) AS num_flights
FROM flights
GROUP BY month
ORDER BY num_flights DESC
LIMIT 1
""")

print(result)


shape: (1, 2)
┌───────┬─────────────┐
│ month ┆ num_flights │
│ ---   ┆ ---         │
│ i64   ┆ u32         │
╞═══════╪═════════════╡
│ 7     ┆ 29425       │
└───────┴─────────────┘


### 2.3 Calculate the on-time performance rate for each carrier

Consider a flight on-time if the departure delay is <= 15 minutes.

In [ ]:
result = ctx.execute("""
SELECT
    carrier,
    100.0 * SUM(CASE WHEN dep_delay <= 15 THEN 1 ELSE 0 END) / COUNT(*) AS on_time_rate
FROM flights
GROUP BY carrier
""")

print(result)

shape: (16, 2)
┌─────────┬──────────────┐
│ carrier ┆ on_time_rate │
│ ---     ┆ ---          │
│ str     ┆ f64          │
╞═════════╪══════════════╡
│ US      ┆ 84.987339    │
│ YV      ┆ 64.725458    │
│ AA      ┆ 82.437594    │
│ FL      ┆ 71.687117    │
│ B6      ┆ 76.537018    │
│ …       ┆ …            │
│ DL      ┆ 83.074205    │
│ UA      ┆ 78.043126    │
│ 9E      ┆ 70.124594    │
│ OO      ┆ 71.875       │
│ VX      ┆ 82.099961    │
└─────────┴──────────────┘


## Exercise 3: Joins

### 3.1 List all flights with their airline names (not just carrier codes)

Show the first 20 flights with carrier code, airline name, flight number, origin, and destination.

In [ ]:
result = ctx.execute("""
SELECT
    f.carrier,
    a.name AS airline_name,
    f.flight,
    f.origin,
    f.dest
FROM flights AS f
JOIN airlines AS a
    ON f.carrier = a.carrier
LIMIT 20
""")

print(result)

shape: (20, 5)
┌─────────┬────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ airline_name           ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                    ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                    ┆ i64    ┆ str    ┆ str  │
╞═════════╪════════════════════════╪════════╪════════╪══════╡
│ UA      ┆ United Air Lines Inc.  ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.  ┆ 1714   ┆ LGA    ┆ IAH  │
│ AA      ┆ American Airlines Inc. ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways        ┆ 725    ┆ JFK    ┆ BQN  │
│ DL      ┆ Delta Air Lines Inc.   ┆ 461    ┆ LGA    ┆ ATL  │
│ …       ┆ …                      ┆ …      ┆ …      ┆ …    │
│ B6      ┆ JetBlue Airways        ┆ 1806   ┆ JFK    ┆ BOS  │
│ UA      ┆ United Air Lines Inc.  ┆ 1187   ┆ EWR    ┆ LAS  │
│ B6      ┆ JetBlue Airways        ┆ 371    ┆ LGA    ┆ FLL  │
│ MQ      ┆ Envoy Air              ┆ 4650   ┆ LGA    ┆ ATL  │
│ B6      ┆ JetBlue Airways        ┆ 343    ┆ EWR    ┆ 

### 3.2 Find the average age of planes for each carrier

Hint: The planes table has a `year` column for manufacture year. Calculate age based on 2013.

In [ ]:
result = ctx.execute("""
SELECT
    f.carrier,
    AVG(2013 - p.year) AS avg_plane_age
FROM flights AS f
JOIN planes AS p
    ON f.tailnum = p.tailnum
GROUP BY f.carrier
""")

print(result)

shape: (16, 2)
┌─────────┬───────────────┐
│ carrier ┆ avg_plane_age │
│ ---     ┆ ---           │
│ str     ┆ f64           │
╞═════════╪═══════════════╡
│ AA      ┆ 25.869426     │
│ FL      ┆ 11.385829     │
│ US      ┆ 9.103663      │
│ F9      ┆ 4.87874       │
│ 9E      ┆ 7.101053      │
│ …       ┆ …             │
│ B6      ┆ 6.686702      │
│ AS      ┆ 3.33662       │
│ EV      ┆ 11.308998     │
│ HA      ┆ 1.548387      │
│ YV      ┆ 9.313758      │
└─────────┴───────────────┘


### 3.3 Find flights that experienced both departure delays and bad weather

Join flights with weather data and find flights where departure delay > 30 minutes and either wind_speed > 20 or precip > 0.1

In [ ]:
result = ctx.execute("""
SELECT
    f.flight,
    f.origin,
    f.dest,
    f.dep_delay,
    w.wind_speed,
    w.precip
FROM flights AS f
JOIN weather AS w
    ON f.origin = w.origin AND f.time_hour = w.time_hour
WHERE f.dep_delay > 30
  AND (w.wind_speed > 20 OR w.precip > 0.1)
""")

print(result)


shape: (3_758, 6)
┌────────┬────────┬──────┬───────────┬────────────┬────────┐
│ flight ┆ origin ┆ dest ┆ dep_delay ┆ wind_speed ┆ precip │
│ ---    ┆ ---    ┆ ---  ┆ ---       ┆ ---        ┆ ---    │
│ i64    ┆ str    ┆ str  ┆ i64       ┆ f64        ┆ i64    │
╞════════╪════════╪══════╪═══════════╪════════════╪════════╡
│ 21     ┆ JFK    ┆ TPA  ┆ 47        ┆ 21.86482   ┆ 0      │
│ 199    ┆ JFK    ┆ LAS  ┆ 116       ┆ 21.86482   ┆ 0      │
│ 4090   ┆ EWR    ┆ JAX  ┆ 39        ┆ 20.71404   ┆ 0      │
│ 4231   ┆ EWR    ┆ IAD  ┆ 40        ┆ 24.16638   ┆ 0      │
│ 1010   ┆ JFK    ┆ BOS  ┆ 33        ┆ 20.71404   ┆ 0      │
│ …      ┆ …      ┆ …    ┆ …         ┆ …          ┆ …      │
│ 2347   ┆ LGA    ┆ ATL  ┆ 41        ┆ 20.71404   ┆ 0      │
│ 5503   ┆ LGA    ┆ PIT  ┆ 97        ┆ 20.71404   ┆ 0      │
│ 135    ┆ JFK    ┆ PHX  ┆ 42        ┆ 21.86482   ┆ 0      │
│ 797    ┆ JFK    ┆ LAX  ┆ 46        ┆ 21.86482   ┆ 0      │
│ 985    ┆ JFK    ┆ BOS  ┆ 45        ┆ 21.86482   ┆ 0      │
└─────

## Exercise 4: Advanced Queries

### 4.1 Find the most popular aircraft types (by number of flights)

Join flights with planes to get manufacturer and model information. Show top 10.

In [ ]:
result = ctx.execute("""
SELECT
    p.manufacturer,
    p.model,
    COUNT(*) AS num_flights
FROM flights AS f
JOIN planes AS p
    ON f.tailnum = p.tailnum
GROUP BY p.manufacturer, p.model
ORDER BY num_flights DESC
LIMIT 10
""")

print(result)


shape: (10, 3)
┌───────────────────────────────┬─────────────────┬─────────────┐
│ manufacturer                  ┆ model           ┆ num_flights │
│ ---                           ┆ ---             ┆ ---         │
│ str                           ┆ str             ┆ u32         │
╞═══════════════════════════════╪═════════════════╪═════════════╡
│ AIRBUS                        ┆ A320-232        ┆ 31278       │
│ EMBRAER                       ┆ EMB-145LR       ┆ 28027       │
│ EMBRAER                       ┆ ERJ 190-100 IGW ┆ 23716       │
│ AIRBUS INDUSTRIE              ┆ A320-232        ┆ 14553       │
│ EMBRAER                       ┆ EMB-145XR       ┆ 14051       │
│ BOEING                        ┆ 737-824         ┆ 13809       │
│ BOMBARDIER INC                ┆ CL-600-2D24     ┆ 11807       │
│ BOEING                        ┆ 737-7H4         ┆ 10389       │
│ BOEING                        ┆ 757-222         ┆ 9150        │
│ MCDONNELL DOUGLAS AIRCRAFT CO ┆ MD-88           ┆ 8932     

### 4.2 Analyze route performance

Find the top 10 routes (origin-destination pairs) with:
- Total number of flights
- Average departure delay
- Percentage of flights delayed more than 30 minutes

Include airport names, not just codes.

In [ ]:
result = ctx.execute("""
SELECT
    f.origin,
    MAX(origin_air.name) AS origin_name,
    f.dest,
    MAX(dest_air.name) AS dest_name,
    COUNT(*) AS total_flights,
    AVG(f.dep_delay) AS avg_dep_delay,
    100.0 * SUM(CASE WHEN f.dep_delay > 30 THEN 1 ELSE 0 END) / COUNT(*) AS pct_delayed_over_30
FROM flights AS f
JOIN airports AS origin_air
    ON f.origin = origin_air.faa
JOIN airports AS dest_air
    ON f.dest = dest_air.faa
GROUP BY f.origin, f.dest
ORDER BY total_flights DESC
LIMIT 10
""")

print(result)


shape: (10, 7)
┌────────┬────────────────┬──────┬────────────────┬───────────────┬───────────────┬────────────────┐
│ origin ┆ origin_name    ┆ dest ┆ dest_name      ┆ total_flights ┆ avg_dep_delay ┆ pct_delayed_ov │
│ ---    ┆ ---            ┆ ---  ┆ ---            ┆ ---           ┆ ---           ┆ er_30          │
│ str    ┆ str            ┆ str  ┆ str            ┆ u32           ┆ f64           ┆ ---            │
│        ┆                ┆      ┆                ┆               ┆               ┆ f64            │
╞════════╪════════════════╪══════╪════════════════╪═══════════════╪═══════════════╪════════════════╡
│ JFK    ┆ John F Kennedy ┆ LAX  ┆ Los Angeles    ┆ 11262         ┆ 8.522508      ┆ 9.829515       │
│        ┆ Intl           ┆      ┆ Intl           ┆               ┆               ┆                │
│ LGA    ┆ La Guardia     ┆ ATL  ┆ Hartsfield     ┆ 10263         ┆ 11.448621     ┆ 12.247881      │
│        ┆                ┆      ┆ Jackson        ┆               ┆         

## Bonus: Compare with Polars

### Choose one of the queries above and implement it using Polars

This will help you understand the relationship between SQL and Polars operations.

In [ ]:
# Example: Let's implement Exercise 2.1 (average delay by origin) in Polars

# SQL version (for reference)
sql_result = ctx.execute("""
    SELECT
        origin,
        AVG(dep_delay) as avg_delay
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY origin
    ORDER BY avg_delay DESC
""")

# Polars version
polars_result = (
    flights
    .filter(pl.col('dep_delay').is_not_null())
    .group_by('origin')
    .agg(pl.col('dep_delay').mean().alias('avg_delay'))
    .sort('avg_delay', descending=True)
)

print("SQL Result:")
print(sql_result)
print("\nPolars Result:")
print(polars_result)

# Now implement one of your own queries in Polars below:
# Your Polars code here

# Polars version of top 10 destinations by number of flights
top_destinations = (
    flights
    .group_by('dest')
    .agg(pl.count().alias('num_flights'))
    .sort('num_flights', descending=True)
    .limit(10)
)

print(top_destinations)

SQL Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘

Polars Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘
shape: (10, 2)
┌──────┬─────────────┐
│ dest ┆ num_flights │
│ ---  ┆ ---         │
│ str  ┆ u32         │
╞══════╪═════════════╡
│ ORD  ┆ 17283       │
│ ATL  ┆ 17215       │
│ LAX  ┆ 16174       │
│ BOS  ┆ 15508       │
│ MCO  ┆ 14082       │
│ CLT  ┆ 14064       │
│ SFO  ┆ 13331       │
│ FLL  ┆ 12055       │
│ MIA  ┆ 11728       │
│ DCA  ┆ 9705        │
└──────┴─────────────┘


/tmp/ipython-input-3684602391.py:35: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias('num_flights'))


Since the SQL query is encountering difficulties, here is the solution using native Polars DataFrame operations to analyze route performance.